# OpenR1 Qwen2.5-0.5B-gsm8k-ReinforcePlusPlus 0424 0508
* num_generations: 8
* num_train_epochs: 1

https://github.com/lzhxmu/CPPO.git


- gpu: T4*1(Colab)
- model: Qwen/Qwen2.5-0.5B
- data: stpete2/openai-gsm8k-part
- method: raftvanilla
- output: Qwen2.5-0.5B-gsm8k-raftvanilla

###### unique setting for cppo in custom_config2.yaml
- metric: smallest
- pruning: 0.5
- allocation: true
  
###### unique setting for raft++ in custom_config2.yaml
- metric: largest
- pruning: 0.5
- allocation: false
- policy_loss: plusplus

## Open-R1
is an open initiative to replicate and extend the techniques behind DeepSeek-R1, a state-of-the-art reasoning model, in a fully transparent and collaborative way:

https://github.com/huggingface/open-r1



In [1]:
import wandb
secret_value = "xxxxx"
wandb.login(key=secret_value)

# save metrics into wandb folder
import os
os.environ["WANDB_DIR"] = "./wandb"
wandb.init(project="250424raft", mode="online")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: stpeteishii to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


import wandb
secret_value = "xxxxx"
wandb.login(key=secret_value)

# save metrics into wandb folder
import os
os.environ["WANDB_DIR"] = "./wandb"
wandb.init(project="250424cppo", mode="online")

In [2]:
!git clone https://github.com/tztechno/forked_CPPO.git
!pip install -e ./forked_CPPO
!pip show forked_CPPO

Cloning into 'forked_CPPO'...
remote: Enumerating objects: 746, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 746 (delta 40), reused 3 (delta 3), pack-reused 667 (from 1)
Receiving objects: 100% (746/746), 3.76 MiB | 37.07 MiB/s, done.
Resolving deltas: 100% (395/395), done.
Obtaining file:///content/forked_CPPO
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/trl.git (to revision 69ad852e5654a77f1695eb4c608906fe0c7e8624) to /tmp/pip-install-kzgmuirj/trl_23a88528ded94fbe803701f4cc72e1bb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-install-kzgmuirj/trl_23a88528ded94fbe803701f4cc72e1bb
  Running command git rev-parse -q --verify 'sha^69ad852e5654a77f1695eb4c608906fe0c7e8624'
  Running command git fetch -q https://github.com/huggingface/trl.git 69ad852e5654a77f1695eb4c608906fe0c7e8624
  Running command git checkout -q 69ad852e5654a

In [3]:
import os
os.chdir('./forked_CPPO')

In [4]:
!ls

asset		   LICENSE   README0.md  recipes  setup.cfg  src
commit_summary.md  Makefile  README.md	 scripts  setup.py


In [5]:
#!pip install flash-attn --no-build-isolation

In [6]:
from pathlib import Path


config_content = """
compute_environment: LOCAL_MACHINE
debug: false

deepspeed_config:
  gradient_clipping: 1.0
  zero3_init_flag: true
  zero_stage: 1
distributed_type: no

downcast_bf16: 'no'
machine_rank: 0
main_training_function: main
mixed_precision: bf16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false
"""

config_path = "custom_config.yaml"
Path(config_path).write_text(config_content)


#------------------------------------------------


config_content2 = """
# Model arguments
model_name_or_path: stpete2/Qwen2.5-0.5B-gsm8k-sft
model_revision: main
torch_dtype: bfloat16
attn_implementation: eager

# Data training arguments
dataset_name: stpete2/openai-gsm8k-part
system_prompt: |
  You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>
  ...
  </think>
  <answer>
  ...
  </answer>
# GRPO trainer config
bf16: true
use_vllm: false
do_eval: false
gradient_accumulation_steps: 2
gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false
hub_model_id: Qwen2.5-0.5B-gsm8k-reinforceplusplus
hub_strategy: every_save
learning_rate: 2.0e-06
log_completions: true
log_level: info
logging_first_step: true
logging_steps: 1
logging_strategy: steps
lr_scheduler_type: cosine
max_prompt_length: 256
max_completion_length: 512
max_steps: -1

num_generations: 4
num_train_epochs: 1


reinforce_variant: plusplus
reinforce_pp_beta: 0.1
use_reward_scaling: true
reward_scaling_factor: 1.0


output_dir: data/Qwen2.5-0.5B-gsm8k-reinforceplusplus
overwrite_output_dir: true
per_device_eval_batch_size: 4
per_device_train_batch_size: 4
push_to_hub: false
report_to:
- wandb
reward_funcs:
- accuracy
- format
- tag_count
reward_weights:
- 1.0
- 1.0
- 1.0
save_strategy: "epoch"
save_total_limit: 1
seed: 42
warmup_ratio: 0.1
"""

config_path2 = "custom_config2.yaml"
Path(config_path2).write_text(config_content2)


#------------------------------------------------


!accelerate launch --config_file custom_config.yaml src/open_r1/grpo_gsm_raft.py \
--config custom_config2.yaml \
--disable_tqdm=False



2025-05-08 06:50:51.513798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746687051.534223    5481 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746687051.540544    5481 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-08 06:50:51.560810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-08 06:50:57,226] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detec

In [9]:
import shutil
from google.colab import files
folder_path = "data"
zip_filename = "data.zip"
shutil.make_archive(folder_path, 'zip', folder_path)
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>